In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
import numpy as np
import sys

In [ ]:
np.random.seed(0)
tf.random.set_seed(3)

# 데이터 로드
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(-1, 28, 28, 1).astype('float32')  / 255
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32')  / 255
# print(x_train[0])
# print(y_train[0])

# one- hot
y_train = to_categorical(y_train)
print(y_train[0])
y_test = to_categorical(y_test)



In [ ]:
# 이미지 보강 클래스 : 기본 이미지를 좌우대칭, 회전 기울기, 이동 등을 통해 이미지의 양을 늘리는 것.
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# 연습
img_gen = ImageDataGenerator(
    rotation_range = 10,
    zoom_range = 0.1,
    shear_range = 0.5,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    horizontal_flip = True,
    vertical_flip = False,
)
augument_size = 100
x_augument = img_gen.flow(np.tile(x_train[0].reshape(28 * 28), 100).reshape(-1, 28, 28, 1),
                          np.zeros(augument_size),
                          batch_size=augument_size,
                          shuffle=False).next()[0]
plt.figure(figsize=(10, 10))
for c in range(100):
  plt.subplot(10, 10, c + 1)
  plt.axis('off')
  plt.imshow(x_augument[c].reshape(28, 28), cmap='gray')
plt.show()

In [ ]:
img_generate = ImageDataGenerator(
    rotation_range = 10,
    zoom_range = 0.1,
    shear_range = 0.5,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    horizontal_flip = False,
    vertical_flip = False
)
augument_size = 30000     # 변형 이미지 3만개
randIdx = np.random.randint(x_train.shape[0], size=augument_size)
x_augment = x_train[randIdx].copy()
y_augment = y_train[randIdx].copy()

x_augument = img_generate.flow(x_augment,
                          np.zeros(augument_size),
                          batch_size=augument_size,
                          shuffle=False).next()[0]

# 원래 이미지에 증식된 이미지를 추가
x_train = np.concatenate((x_train, x_augment))
y_train = np.concatenate((y_train, y_augment))
print(x_train.shape)

In [ ]:
# 모델 생성
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), input_shape=(28, 28, 1), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), input_shape=(28, 28, 1), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(units=10, activation='softmax'),
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

In [ ]:
# 조기 종료 조건 callbacks 모듈의 함수
early_stop = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(x_train, y_train, validation_split=0.2, epochs=100, batch_size=64, verbose=2, callbacks=[early_stop])
print('Accuracy : %.3f' %(model.evaluate(x_test, y_test)[1]))

In [ ]:
print('accuracy : %.3f'%(model.evaluate(x_test, y_test)[1]))

# 시각화
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], marker='o', c='red', label='acc')
plt.plot(history.history['val_accuracy'], marker='s', c='blue', label='val_acc')
plt.xlabel('epochs')
plt.ylim(0.5, 1)
plt.legend(loc='lower right')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], marker='o', c='red', label='loss')
plt.plot(history.history['val_loss'], marker='s', c='blue', label='val_loss')
plt.xlabel('epochs')
plt.legend(loc='upper right')
plt.show()
